# **Stacking from scratch**

Berikan penjelasan terkait Stacking!

Source : https://www.geeksforgeeks.org/machine-learning/stacking-in-machine-learning/
Youtube : https://www.youtube.com/watch?v=a4IS1Ai7GCI

In [18]:
# Import Libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC 
from sklearn.metrics import accuracy_score

**Pendefinisian class stack**

Bertujuan untuk membuat kerangka kerja ensemble learning berbasis stacking dan blending secara modular dan fleksibel. Class ini memungkinkan pengguna untuk menggabungkan beberapa model dasar (base learners) dan satu model meta (final estimator) untuk menghasilkan prediksi akhir yang lebih akurat. Dengan mengatur parameter seperti metode cross-validation, penggunaan blending, serta paralelisasi proses, class ini memberikan kontrol penuh kepada pengguna dalam menerapkan teknik stacking sesuai kebutuhan. Kegunaan utamanya adalah sebagai alat bantu untuk mengimplementasikan ensemble model dari nol (from scratch) tanpa tergantung pada fungsi otomatis dari pustaka seperti scikit-learn, sehingga cocok untuk pembelajaran konsep maupun eksperimen lanjutan dalam machine learning.

Buat class bernama Stack yang berisi <br>

Attribute:


Method:


In [9]:
# Definisikan class stack
class Stack(BaseEstimator, ClassifierMixin):

    def __init__(self, base_learners=None, final_estimator=None, cv=5):
        if base_learners is None:
            self.base_learners = [('lr', LogisticRegression(random_state=42))]
        else:
            self.base_learners = base.learners

        if final_estimator is None:
            self.final_estimator = LogisticsRegression(random_state=42)
        else:
            self.final_estimator = final_estimator

        self.cv = cv
        self.fitted_base_learners = []

    def fit(self, X, y):
        kf = KFold(n_splits=self.cv, shuffle=True, random_state=42)
        meta_features = np.zeros((X.shape[0], len(self.base_learners)))
        self.fitted_base_learners = []

        for i, (name, model) in enumerate(self.base_learners):
            clone_model = copy.deepcopy(model)
            self.fitted_base_learnenrs.append(clone_model)
            oof_predictions = np.zeros((X.Shape[0],))

            for train_idx, val_idx in kf.split(X, y):
                X_train_fold, X_val_fold = X[train_idx], X[val_idx]
                y_train_fold, y_val_fold = y[train_idx], y[val_idx]

                clone_model.fit(X_train_fold, y_train_fold)

                oof_predictions[val_idx] = clone_model.predict(X_val_fold)

            meta_features[:, i] = oof_predictions

            clone_model.fit(X, y)

        self.final_estimator.fit(meta_features, y)
        return self

    def predict(self, X):
        base_predictions = []
        
        for name, model in zip(self.base_learners, self.fitted_base_learners):
            base_predictions.append(model.predict(X))

        meta_features = np.column_stack(base_predicitions)
        return self.final_estimator.predict(meta_features)

### **Upload datasets**

In [10]:
# Load dataset iris dari sklearn
from sklearn.datasets import load_iris
iris = load_iris(as_frame=True)
glass_df = iris.frame

# Pisahkan fitur dan target
X1 = glass_df[iris.feature_names].values
y1 = glass_df['target'].values
# Encode target (optional, untuk case multiclass stacking) -> tidak diperlukan

# Split data menjadi train dan test
X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, random_state=0)

# **Model training and evaluation of the obtained results**
Both in the case of classification and regression, stacking and blending showed the same and not the best results. As a rule, this situation occurs for two reasons: given that metadata is based on predictions of basic models, the presence of weak basic models can reduce the accuracy of stronger ones which will reduce the final prediction as a whole. Also a small amount of training data often leads to overfitting which in turn reduces the accuracy of predictions.

In this case the problem can be partially solved by setting stack_method='predict_proba' when each basic classifier outputs class membership probabilities instead of the classes themselves which can help increase accuracy in the case of non-mutually exclusive classes. Also this method works better with noise in the data. As you can see this method has significantly increased the accuracy of the model. With the right selection of models and hyperparameters the accuracy will be even higher.

Most often stacking shows slightly better results than blending due to the use of k-fold cross-validation but usually the difference is noticeable only on a large amount of data.

In [11]:
# StackingClassifier dan Blending Classifier


**StackingClassifier (scikit-learn)**


* Merupakan implementasi resmi stacking untuk klasifikasi dalam scikit-learn.

* Mempermudah proses ensemble dengan base learners dan final estimator (meta learner) dalam satu objek.

* Sudah menangani cross-validation secara internal sehingga mengurangi risiko data leakage.

* Mendukung parameter passthrough=True jika ingin menggabungkan fitur asli dengan meta-features.

* Cocok untuk digunakan dalam pipeline dan produksi karena stabil dan teruji.

In [19]:
# StackingClassifier (scikit-learn)
from sklearn.ensemble import StackingClassifier

# sk_estimators
sk_estimators = [
    ('knn', KNeighborsClassifier(n_neighbors=5)),
    ('dt', DecisionTreeClassifier(max_depth=5, random_state=42)),
    ('svc', SVC(kernel='linear', probability=True, random_state=42))
]

meta_learner = LogisticRegression(solver='lbfgs', multi_class='auto', random_state=42)

# stack_methods
stack_methods = ['predict_proba', 'predict']

result = {}

for i, method in enumerate(stack_methods):
    print(method)

    current_stack = StackingClassifier(
    estimators = sk_estimators,
    final_estimator = meta_learner,
    cv = 5,
    passthrough = False
    )

    current_stack.fit(X1_train, y1_train)

    preds = current_stack.predict(X1_test)
    accuracy = accuracy_score(y1_test, preds)
    result[method] = accuracy

    print(f"Metode {method} akurasi: {accuracy}")

print("Ringkasan akurasi Stacking Classifier")
print(result)

predict_proba
Metode predict_proba akurasi: 0.9736842105263158
predict
Metode predict akurasi: 0.9736842105263158
Ringkasan akurasi Stacking Classifier
{'predict_proba': 0.9736842105263158, 'predict': 0.9736842105263158}
